**Importação das bibliotecas**

In [4]:
import numpy as np
from scipy.linalg import lu

**Funções Auxiliares**

In [5]:
def substituicao_direta(L, b):
  """
  Resolve o sistema triangular inferior Ly = b para y (Forward Substitution).
  L tem 1s na diagonal - Método Doolittle.
  """
  n = L.shape[0]
  y = np.zeros(n, dtype=float)

  for i in range(n):
    sum_j = 0
    for j in range(i):
      sum_j += L[i, j] * y[j]
      y[i] = b[i] - sum_j

    return y

def substituicao_reversa(U, y):
  """
  Resolve o sistema triangular superior Ux = y para x (Backward substitution).
  """
  n = U.shape[0]
  x = np.zeros(n, dtype=float)

  for i in range(n - 1, -1, -1):
    sum_j = 0
    for j in range(i + 1, n):
      sum_j += U[i, j] * x[j]

    if np.isclose(U[i, i], 0):
      raise ValueError("Matriz U é singular, o sistema pode não ter solução única.")

    x[i] = (y[i] - sum_j) / U[i, i]

  return x

**Funções Principais**

In [6]:
def resolve_lu(A_in, b_in):
  """
  Implementa as três etapas do processo de decomposição LU
  """
  A = np.array(A_in, dtype=float)
  b = np.array(b_in, dtype=float)

  P, L, U = lu(A)

  Pb = P @ b
  y = substituicao_direta(L, Pb)
  x = substituicao_reversa(U, y)

  return P, L, U, y, x

def resolver_por_gauss(A_in, b_in):
  """ Resolve o sistema Ax=b usando a implementação otimizada de eliminação Gaussiana do NumPy. """
  A = np.array(A_in, dtype=float)
  b = np.array(b_in, dtype=float)

  x = np.linalg.solve(A, b)
  return x

**Exercício 1 (a) A**

In [7]:
A1a = [[2, 1], [1, 2]]
b1a = [1, 2]

P, L, U, y, x = resolve_lu(A1a, b1a)
# -------- Exercício 1 (a) --------
print(f"Matriz P:\n{P}\n")
print(f"Matriz L (diagonal de 1s):\n{L}\n")
print(f"Matriz U:\n{U}\n")
print(f"Solução intermediária y (de Ly=Pb):\n{y}\n")
print(f"Solução final x (de Ux=y):\n{x}")
# Verificação:
print(f"Verificação (Ax): {np.array(A1a) @ x}")
print(f"Solução esperada: [0, 1]")

Matriz P:
[[1. 0.]
 [0. 1.]]

Matriz L (diagonal de 1s):
[[1.  0. ]
 [0.5 1. ]]

Matriz U:
[[2.  1. ]
 [0.  1.5]]

Solução intermediária y (de Ly=Pb):
[0. 0.]

Solução final x (de Ux=y):
[0. 0.]
Verificação (Ax): [0. 0.]
Solução esperada: [0, 1]
